In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 15
l = 40
num_filter = 32
compression = 0.5
dropout_rate = 0.5

In [6]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 19s 0us/step


In [0]:
from keras.layers import LeakyReLU

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = LeakyReLU(alpha=0.1)(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = LeakyReLU(alpha=0.1)(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = LeakyReLU(alpha=0.1)(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 64
dropout_rate = 0.25
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [27]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_237 (Conv2D)             (None, 32, 32, 64)   1728        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_237 (BatchN (None, 32, 32, 64)   256         conv2d_237[0][0]                 
__________________________________________________________________________________________________
leaky_re_lu_53 (LeakyReLU)      (None, 32, 32, 64)   0           batch_normalization_237[0][0]    
__________________________________________________________________________________________________
conv2d_238

Total params: 3,093,578
Trainable params: 3,069,450
Non-trainable params: 24,128
__________________________________________________________________________________________________


In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [30]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=20,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 614s 12ms/step - loss: 0.7755 - acc: 0.7262 - val_loss: 1.0389 - val_acc: 0.6866
Epoch 2/20
19072/50000 [==========>...................] - ETA: 5:55 - loss: 0.6838 - acc: 0.7580

50000/50000 [==============================] - 615s 12ms/step - loss: 0.6620 - acc: 0.7664 - val_loss: 0.9517 - val_acc: 0.7199
Epoch 3/20
45184/50000 [==========================>...] - ETA: 55s - loss: 0.6036 - acc: 0.7879

50000/50000 [==============================] - 615s 12ms/step - loss: 0.6000 - acc: 0.7896 - val_loss: 0.9136 - val_acc: 0.7288
Epoch 4/20
50000/50000 [==============================] - 615s 12ms/step - loss: 0.5474 - acc: 0.8076 - val_loss: 1.0810 - val_acc: 0.6984
Epoch 5/20
 2944/50000 [>.............................] - ETA: 9:00 - loss: 0.5113 - acc: 0.8162

50000/50000 [==============================] - 615s 12ms/step - loss: 0.5023 - acc: 0.8233 - val_loss: 0.8583 - val_acc: 0.7414
Epoch 6/20
39040/50000 [======================>.......] - ETA: 2:06 - loss: 0.4661 - acc: 0.8367

50000/50000 [==============================] - 615s 12ms/step - loss: 0.4675 - acc: 0.8362 - val_loss: 0.7163 - val_acc: 0.7943
Epoch 7/20
50000/50000 [==============================] - 615s 12ms/step - loss: 0.4343 - acc: 0.8484 - val_loss: 1.3759 - val_acc: 0.7060
Epoch 8/20
 1536/50000 [..............................] - ETA: 9:18 - loss: 0.4045 - acc: 0.8555

...

50000/50000 [==============================] - 614s 12ms/step - loss: 0.2932 - acc: 0.8974 - val_loss: 0.7961 - val_acc: 0.7975
Epoch 15/20
38272/50000 [=====================>........] - ETA: 2:14 - loss: 0.2745 - acc: 0.9030

50000/50000 [==============================] - 614s 12ms/step - loss: 0.2768 - acc: 0.9027 - val_loss: 0.5790 - val_acc: 0.8439
Epoch 16/20
50000/50000 [==============================] - 614s 12ms/step - loss: 0.2618 - acc: 0.9075 - val_loss: 0.8553 - val_acc: 0.7985
Epoch 17/20
 1280/50000 [..............................] - ETA: 9:21 - loss: 0.2801 - acc: 0.8969

50000/50000 [==============================] - 614s 12ms/step - loss: 0.2541 - acc: 0.9087 - val_loss: 0.7178 - val_acc: 0.8221
Epoch 18/20
38272/50000 [=====================>........] - ETA: 2:14 - loss: 0.2324 - acc: 0.9180

50000/50000 [==============================] - 614s 12ms/step - loss: 0.2358 - acc: 0.9162 - val_loss: 0.6095 - val_acc: 0.8459
Epoch 19/20
50000/50000 [==============================] - 614s 12ms/step - loss: 0.2245 - acc: 0.9202 - val_loss: 0.7369 - val_acc: 0.8219
Epoch 20/20
 1280/50000 [..............................] - ETA: 9:20 - loss: 0.2256 - acc: 0.9273

50000/50000 [==============================] - 614s 12ms/step - loss: 0.2149 - acc: 0.9239 - val_loss: 0.8045 - val_acc: 0.8075


In [31]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 48s 5ms/step
Test loss: 0.8045162917375565
Test accuracy: 0.8075


Note : By Just Changing the Activation function to LeakyRelu and using 32 filter, able to achieve 80% Validation accuracy.

In [32]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')